In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib as plt
import random
from citipy import citipy
from datetime import datetime 
from config import weather_api_key
import requests
from config import weather_api_key
import time

In [ ]:
#Create a set of random latitude and longitude combinations.
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [ ]:
#Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [ ]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination. 
for coordinate in coordinates: 
    city =citipy.nearest_city(coordinate[0], coordinate[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # If the city is unique, then we will add it to the cities list. 
    if city_name not in cities:
        cities.append(city_name)
    
    #print(f'{city_name}, {country_name}')
        
#print the city count to confirm sufficient count.
len(cities)

In [ ]:
# Starting URL for weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#Create an empty list to hold weather data.
city_data = []

#Print the beginning of the logging
print("Beginning Data Retrieval   ")
print("---------------------------")

#Create counters.
record_count = 1
set_count = 1

In [ ]:
# Loop through city lists
for i, city in enumerate(cities):
    
    #Group cities in sets of 50
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count += 1
        
    #Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1
    
# Make an API request for each city
    try:
        
        #Parse the JSON data
        city_weather = requests.get(city_url).json()
        
        #Compile needed data
        city_city = city.title()
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']
        
        #Fill city data list
        city_data.append({"City": city_city,
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max_Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Description": city_desc
                         })

    #If an error occurs
    except:
        print("City not found. Skipping")
        pass
    
    #Complete data retriveal
print("------------------------------")
print("Data Retriveal Complete       ")
print("------------------------------")



In [ ]:
len(city_data)

In [ ]:
#Creating a DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
#Create csv file of the DataFrame
csv_weather_file = "WeatherPy_Database.csv"

#Export CSV file
city_data_df.to_csv(csv_weather_file, index_label="City_ID")
